In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

from xai_agg import *

import dill

# Disable pandas column limit
pd.set_option('display.max_columns', None)

2025-01-22 09:14:22.251639: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 09:14:22.277744: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data loading and preprocessing

In [2]:
raw = pd.read_csv("../data/PAKDD2010.tsv", sep="\t", encoding="unicode_escape", header=None)

raw.columns = [
    "ID_CLIENT", "CLERK_TYPE", "PAYMENT_DAY", "APPLICATION_SUBMISSION_TYPE", "QUANT_ADDITIONAL_CARDS", 
    "POSTAL_ADDRESS_TYPE", "SEX", "MARITAL_STATUS", "QUANT_DEPENDANTS", "UNKNOWN",
    "STATE_OF_BIRTH", "CITY_OF_BIRTH", "NACIONALITY", "RESIDENCIAL_STATE", "RESIDENCIAL_CITY", 
    "RESIDENCIAL_BOROUGH", "FLAG_RESIDENCIAL_PHONE", "RESIDENCIAL_PHONE_AREA_CODE", "RESIDENCE_TYPE", 
    "MONTHS_IN_RESIDENCE", "FLAG_MOBILE_PHONE", "FLAG_EMAIL", "PERSONAL_MONTHLY_INCOME", "OTHER_INCOMES", 
    "FLAG_VISA", "FLAG_MASTERCARD", "FLAG_DINERS", "FLAG_AMERICAN_EXPRESS", "FLAG_OTHER_CARDS", 
    "QUANT_BANKING_ACCOUNTS", "QUANT_SPECIAL_BANKING_ACCOUNTS", "PERSONAL_ASSETS_VALUE", "QUANT_CARS", 
    "COMPANY", "PROFESSIONAL_STATE", "PROFESSIONAL_CITY", "PROFESSIONAL_BOROUGH", "FLAG_PROFESSIONAL_PHONE", 
    "PROFESSIONAL_PHONE_AREA_CODE", "MONTHS_IN_THE_JOB", "PROFESSION_CODE", "OCCUPATION_TYPE", 
    "MATE_PROFESSION_CODE", "EDUCATION_LEVEL", "FLAG_HOME_ADDRESS_DOCUMENT", "FLAG_RG", "FLAG_CPF", 
    "FLAG_INCOME_PROOF", "PRODUCT", "FLAG_ACSP_RECORD", "AGE", "RESIDENCIAL_ZIP_3", "PROFESSIONAL_ZIP_3", 
    "TARGET_LABEL_BAD"
]

raw.drop(columns=["ID_CLIENT", "CLERK_TYPE", "QUANT_ADDITIONAL_CARDS"], inplace=True)

display(raw.head())
display(raw.info())

,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,UNKNOWN,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,FLAG_RESIDENCIAL_PHONE,RESIDENCIAL_PHONE_AREA_CODE,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_MOBILE_PHONE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OTHER_CARDS,QUANT_BANKING_ACCOUNTS,QUANT_SPECIAL_BANKING_ACCOUNTS,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,PROFESSIONAL_CITY,PROFESSIONAL_BOROUGH,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,EDUCATION_LEVEL,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD
0,5,Web,1,F,6,1,0,RN,Assu,1,RN,Santana do Matos,Centro,Y,105,1.0,15.0,N,1,900.0,0.0,1,1,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9.0,4.0,NaN,NaN,0,0,0,0,1,N,32,595,595,1
1,15,Carga,1,F,2,0,0,RJ,rio de janeiro,1,RJ,RIO DE JANEIRO,CAMPO GRANDE,Y,20,1.0,1.0,N,1,750.0,0.0,0,0,0,0,0,0,0,0.0,0,Y,,NaN,NaN,N,,0,11.0,4.0,11.0,NaN,0,0,0,0,1,N,34,230,230,1
2,5,Web,1,F,2,0,0,RN,GARANHUNS,1,RN,Parnamirim,Boa Esperanca,Y,105,1.0,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,11.0,NaN,NaN,NaN,0,0,0,0,1,N,27,591,591,0
3,20,Web,1,F,2,0,0,PE,CABO,1,PE,CABO,PONTE DOS CARVALHOS,N,,NaN,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,NaN,NaN,NaN,NaN,0,0,0,0,1,N,61,545,545,0
4,10,Web,1,M,2,0,0,RJ,RIO DE JANEIRO,1,RJ,Rio de Janeiro,Santa Cruz,Y,20,1.0,12.0,N,1,1200.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9.0,5.0,NaN,NaN,0,0,0,0,1,N,48,235,235,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 51 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   PAYMENT_DAY                     50000 non-null  int64  
 1   APPLICATION_SUBMISSION_TYPE     50000 non-null  object 
 2   POSTAL_ADDRESS_TYPE             50000 non-null  int64  
 3   SEX                             50000 non-null  object 
 4   MARITAL_STATUS                  50000 non-null  int64  
 5   QUANT_DEPENDANTS                50000 non-null  int64  
 6   UNKNOWN                         50000 non-null  int64  
 7   STATE_OF_BIRTH                  50000 non-null  object 
 8   CITY_OF_BIRTH                   50000 non-null  object 
 9   NACIONALITY                     50000 non-null  int64  
 10  RESIDENCIAL_STATE               50000 non-null  object 
 11  RESIDENCIAL_CITY                50000 non-null  object 
 12  RESIDENCIAL_BOROUGH             

None

In [3]:
# Taking a subset of columns:
numeric_features = [
    "QUANT_DEPENDANTS", "RESIDENCIAL_PHONE_AREA_CODE", "MONTHS_IN_RESIDENCE", "PERSONAL_MONTHLY_INCOME", "QUANT_CARS", "PROFESSIONAL_PHONE_AREA_CODE",
    "MONTHS_IN_THE_JOB", "RESIDENCIAL_ZIP_3", "PROFESSIONAL_ZIP_3", "AGE"
]

categorical_features = [
    "SEX", "MARITAL_STATUS", "EDUCATION_LEVEL", "FLAG_RESIDENCIAL_PHONE",
    "FLAG_MOBILE_PHONE", "COMPANY", "FLAG_PROFESSIONAL_PHONE", "OCCUPATION_TYPE"
]


features = numeric_features + categorical_features
preprocessed_data = raw[features + ["TARGET_LABEL_BAD"]]

# Handling missing values. For numeric features, we will use the median value. For categorical features, we will use the mode.
# Handle invalid values:
preprocessed_data.replace(r'^\s*$', np.nan, regex=True, inplace=True)
preprocessed_data.replace(r'(?i)\bnull\b', np.nan, regex=True, inplace=True)
preprocessed_data.replace("#DIV/0!", np.nan, inplace=True)

# Convert all numeric features to float.
for feature in numeric_features:
    preprocessed_data[feature] = preprocessed_data[feature].astype(float)

for feature in numeric_features:
    preprocessed_data[feature] = preprocessed_data[feature].fillna(preprocessed_data[feature].median())

for feature in categorical_features:
    preprocessed_data[feature] = preprocessed_data[feature].fillna(preprocessed_data[feature].mode().values[0])

    # if datatype is not object, convert to int
    if preprocessed_data[feature].dtype != 'object':
        preprocessed_data[feature] = preprocessed_data[feature].astype(int)

# Handling categorical features
# One-hot encoding
preprocessed_data = pd.get_dummies(preprocessed_data, columns=categorical_features, drop_first=True, dtype='int64')

# Stratified sampling
preprocessed_data, _ = train_test_split(preprocessed_data, test_size=0.94, stratify=preprocessed_data['TARGET_LABEL_BAD'], random_state=42)

display(preprocessed_data.head())
display(preprocessed_data.info())


,QUANT_DEPENDANTS,RESIDENCIAL_PHONE_AREA_CODE,MONTHS_IN_RESIDENCE,PERSONAL_MONTHLY_INCOME,QUANT_CARS,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,AGE,TARGET_LABEL_BAD,SEX_M,SEX_N,MARITAL_STATUS_1,MARITAL_STATUS_2,MARITAL_STATUS_3,MARITAL_STATUS_4,MARITAL_STATUS_5,MARITAL_STATUS_6,MARITAL_STATUS_7,EDUCATION_LEVEL_1,EDUCATION_LEVEL_2,EDUCATION_LEVEL_3,EDUCATION_LEVEL_4,EDUCATION_LEVEL_5,FLAG_RESIDENCIAL_PHONE_Y,COMPANY_Y,FLAG_PROFESSIONAL_PHONE_Y,OCCUPATION_TYPE_1,OCCUPATION_TYPE_2,OCCUPATION_TYPE_3,OCCUPATION_TYPE_4,OCCUPATION_TYPE_5
22300,0.0,68.0,12.0,276.0,0.0,66.0,0.0,565.0,565.0,56.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
36609,0.0,67.0,5.0,500.0,0.0,66.0,0.0,749.0,749.0,27.0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
38682,0.0,107.0,10.0,350.0,0.0,66.0,0.0,607.0,607.0,61.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
26159,0.0,47.0,5.0,440.0,1.0,47.0,0.0,855.0,855.0,33.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0
13415,2.0,41.0,8.0,350.0,1.0,66.0,0.0,840.0,840.0,57.0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 22300 to 40119
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   QUANT_DEPENDANTS              3000 non-null   float64
 1   RESIDENCIAL_PHONE_AREA_CODE   3000 non-null   float64
 2   MONTHS_IN_RESIDENCE           3000 non-null   float64
 3   PERSONAL_MONTHLY_INCOME       3000 non-null   float64
 4   QUANT_CARS                    3000 non-null   float64
 5   PROFESSIONAL_PHONE_AREA_CODE  3000 non-null   float64
 6   MONTHS_IN_THE_JOB             3000 non-null   float64
 7   RESIDENCIAL_ZIP_3             3000 non-null   float64
 8   PROFESSIONAL_ZIP_3            3000 non-null   float64
 9   AGE                           3000 non-null   float64
 10  TARGET_LABEL_BAD              3000 non-null   int64  
 11  SEX_M                         3000 non-null   int64  
 12  SEX_N                         3000 non-null   int64  
 13  MAR

None

# Fitting classifier

In [4]:
y = preprocessed_data["TARGET_LABEL_BAD"]
X = preprocessed_data.drop(columns='TARGET_LABEL_BAD')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')

Accuracy: 0.7466666666666667
ROC AUC: 0.5158014399393709


# Experiments

# Evaluating Old and New Metric Sets

### Execution

In [7]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    metrics_sets=[
        ['complexity', 'sensitivity_spearman', 'faithfulness_corr'],
        ['nrc', 'sensitivity_spearman', 'faithfulness_corr']
    ],
    n_instances=5
)

metadata["description"] = "compares entropy complexity with nrc metric sets"
metadata["dataset"] = "pakdd2010"

with open('pickles/pakdd2010/COMPARE_entropy-nrc_metricsets_wsum-topsis.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)


Selected indexes: [15614 47126 44241 37715  6457]
Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2286 - val_loss: 1.1039
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - loss: 1.3691 - val_loss: 1.0574
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - loss: 1.2085 - val_loss: 1.0056
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 1.1842 - val_loss: 0.9547
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - loss: 1.0565 - val_loss: 0.9099
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 611us/step - loss: 1.1400 - val_loss: 0.8743
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 1.0374 - val_loss: 0.8471
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - loss: 0.9682 - val_loss: 0.8265
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.8775 - val_loss: 0.8106
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 573us/step - loss: 1.1158 - val_loss: 0.7978
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - loss: 1.0598 - val_loss: 

### Analysis

In [17]:
with open('pickles/pakdd2010/COMPARE_entropy-nrc_metricsets_wsum-topsis.pkl', 'rb') as f:
    exp = dill.load(f)

In [18]:
for i, metrics in enumerate([['complexity', 'sensitivity_spearman', 'faithfulness_corr'], 
                             ['nrc', 'sensitivity_spearman', 'faithfulness_corr']]):
    print(f"With metrics: {metrics}\n")
    display(exp.results[i])
    wca = count_worst_case_avoidances(exp.results[i], [False, True, True], 1)
    print(f"Worst case avoidances:\n\t- for all metrics: {wca[0]}\n\t- for 2/3 metrics: {wca[1]}")
    print("AVG:")
    display(get_expconfig_mean_results(exp, i))
    print("\n")
    print("Avg rank:")
    display(get_average_metric_rank(exp.results[i], [False, True, True]))

With metrics: ['complexity', 'sensitivity_spearman', 'faithfulness_corr']



[                        complexity  sensitivity_spearman  faithfulness_corr
 LimeWrapper               2.569554              0.832845           0.065883
 ShapTabularTreeWrapper    2.610179              0.969886           0.482561
 AnchorWrapper             1.276633              0.316074           0.856008
 AggregateExplainer        2.662932              0.899707           0.431864,
                         complexity  sensitivity_spearman  faithfulness_corr
 LimeWrapper               2.605060              0.851540           0.052526
 ShapTabularTreeWrapper    2.027555              0.969996           0.250255
 AnchorWrapper             1.609346              0.569517           0.019496
 AggregateExplainer        2.480156              0.891349           0.017006,
                         complexity  sensitivity_spearman  faithfulness_corr
 LimeWrapper               2.562406              0.862683           0.246111
 ShapTabularTreeWrapper    2.382450              0.980426           0.8397

Worst case avoidances:
	- for all metrics: 1
	- for 2/3 metrics: 5
AVG:


,complexity,sensitivity_spearman,faithfulness_corr
AggregateExplainer,2.560851,0.902419,0.341503
AnchorWrapper,1.320434,0.425198,0.289867
LimeWrapper,2.587150,0.841290,0.292162
ShapTabularTreeWrapper,2.441527,0.973544,0.526287




Avg rank:


...

[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             47.184995              0.876979           0.483840
 ShapTabularTreeWrapper  52.863017              0.962688           0.818997
 AnchorWrapper           53.320475              0.405945           0.758160
 AggregateExplainer      50.157699              0.904985           0.483425,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             43.664671              0.852456           0.541449
 ShapTabularTreeWrapper  45.859253              0.977598           0.195383
 AnchorWrapper           44.171957              0.554773           0.459947
 AggregateExplainer      49.506777              0.866606           0.182232,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             48.863019              0.815579           0.212933
 ShapTabularTreeWrapper  49.522242              0.977011           0.914604
 AnchorWra

Worst case avoidances:
	- for all metrics: 3
	- for 2/3 metrics: 4
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,50.145223,0.900740,0.407161
AnchorWrapper,48.583024,0.446059,0.384281
LimeWrapper,47.422547,0.848248,0.395031
ShapTabularTreeWrapper,51.135102,0.973705,0.615147




Avg rank:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,2.6,2.0,3.2
AnchorWrapper,2.2,4.0,2.6
LimeWrapper,1.6,3.0,2.6
ShapTabularTreeWrapper,3.6,1.0,1.6


### Discussion

Metric set with NRC performed better; it avoided the worst case for all metrics in 3/5 of the instances, whereas the metric set with the original entropy metric only did it for 1/5 of the instances. The NRC metric set also had a better average performance.

# Evaluating Rank aggregation algorithms
### Execution

In [8]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    aggregation_algs=["wsum", "w_bordafuse", "w_condorcet"],
    n_instances=5
)

metadata["description"] = "compares wsum, w_bordafuse, w_condorcet aggregation algorithms"
metadata["dataset"] = "pakdd2010"

with open('pickles/pakdd2010/COMPARE_wsum-w_bordafuse-w_condorcet.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)

Selected indexes: [2357 8478 9293 6595  425]
Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.3548 - val_loss: 1.0998
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - loss: 1.2071 - val_loss: 1.0517
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - loss: 1.1659 - val_loss: 1.0005
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - loss: 1.2656 - val_loss: 0.9536
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step - loss: 1.1012 - val_loss: 0.9160
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - loss: 0.9915 - val_loss: 0.8871
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss: 0.9771 - val_loss: 0.8649
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - loss: 1.1360 - val_loss: 0.8474
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - loss: 1.0466 - val_loss: 0.8326
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 607us/step - loss: 0.9669 - val_loss: 0.8196
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 583us/step - loss: 0.9819 - val_loss: 0.808

### Analysis

In [19]:
with open('pickles/pakdd2010/COMPARE_wsum-w_bordafuse-w_condorcet.pkl', 'rb') as f:
    exp = dill.load(f)

In [20]:
for i, method in enumerate(["wsum", "w_bordafuse", "w_condorcet"]):
    print(f"{method}:\n")
    display(exp.results[i])
    wca = count_worst_case_avoidances(exp.results[i], [False, True, True], 1)
    print(f"Worst case avoidances:\n\t- for all metrics: {wca[0]}\n\t- for 2/3 metrics: {wca[1]}")
    print("AVG:")
    display(get_expconfig_mean_results(exp, i))
    print("\n")
    print("Avg rank:")
    display(get_average_metric_rank(exp.results[i], [False, True, True]))

wsum:



[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             47.935318              0.828226           0.824736
 ShapTabularTreeWrapper  44.477749              0.962578           0.827157
 AnchorWrapper           66.634771              0.667473           0.932455
 AggregateExplainer      44.134919              0.912097           0.800706,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             54.358657              0.859384           0.128297
 ShapTabularTreeWrapper  54.728233              0.987697           0.157754
 AnchorWrapper           41.832701              0.543240           0.411779
 AggregateExplainer      51.891224              0.932075           0.366356,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             50.333571              0.778079           0.915059
 ShapTabularTreeWrapper  54.245374              0.976019           0.203481
 AnchorWra

Worst case avoidances:
	- for all metrics: 2
	- for 2/3 metrics: 5
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,50.031571,0.916877,0.405243
AnchorWrapper,49.312625,0.566701,0.563702
LimeWrapper,51.510224,0.823248,0.506196
ShapTabularTreeWrapper,53.665743,0.968278,0.365901




Avg rank:


...

[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             49.865136              0.824340           0.647248
 ShapTabularTreeWrapper  44.477749              0.960779           0.785881
 AnchorWrapper           66.634771              0.708104           0.826956
 AggregateExplainer      79.701933              0.744428           0.353658,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             51.446662              0.833944           0.552006
 ShapTabularTreeWrapper  54.728233              0.987239           0.123457
 AnchorWrapper           42.448236              0.481225           0.226242
 AggregateExplainer      79.701933              0.903006           0.397662,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             50.107873              0.842632           0.860774
 ShapTabularTreeWrapper  54.245374              0.977819           0.058266
 AnchorWra

Worst case avoidances:
	- for all metrics: 0
	- for 2/3 metrics: 4
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,79.701933,0.824062,0.336692
AnchorWrapper,46.730021,0.497732,0.408699
LimeWrapper,51.493273,0.844333,0.565567
ShapTabularTreeWrapper,53.665743,0.966923,0.270182




Avg rank:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,4.0,2.8,2.8
AnchorWrapper,1.4,4.0,2.0
LimeWrapper,2.2,2.2,2.0
ShapTabularTreeWrapper,2.4,1.0,3.2


### Discussion

Once again, w_sum performed better. Not much to say, it avoided the worst case for all metrics the most, and had better average performance, specially on the NRC metric.

# Evaluating MCDM Algs

### Execution

In [7]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    mcdm_algs=[pymcdm.methods.TOPSIS(), pymcdm.methods.COPRAS(),
               pymcdm.methods.PROMETHEE_II(preference_function="usual"),
               pymcdm.methods.ARAS(), pymcdm.methods.COCOSO(),
               pymcdm.methods.CODAS(), pymcdm.methods.EDAS(), pymcdm.methods.MABAC()],
    n_instances=5
)

metadata["description"] = "compares TOPSIS, COPRAS, PROMETHEE_II, ARAS, COCOSO, CODAS, EDAS, MABAC MCDM algorithms"
metadata["dataset"] = "pakdd2010"

with open('pickles/pakdd2010/COMPARE_mcdm_algs.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)

Selected indexes: [23842 38378 46082  8797 38583]
Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.3397 - val_loss: 1.0919
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1967 - val_loss: 1.0443
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 1.1476 - val_loss: 0.9978
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - loss: 1.1510 - val_loss: 0.9551
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 1.1248 - val_loss: 0.9174
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step - loss: 1.0930 - val_loss: 0.8879
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 688us/step - loss: 0.9601 - val_loss: 0.8655
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 0.9785 - val_loss: 0.8481
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 679us/step - loss: 0.9381 - val_loss: 0.8339
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 1.0723 - val_loss: 0.8222
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step - loss: 1.0416 - val_loss: 0.

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.


	 Running instance 38378
	 Running instance 46082
	 Running instance 8797
	 Running instance 38583


### Analysis

In [21]:
with open('pickles/pakdd2010/COMPARE_mcdm_algs.pkl', 'rb') as f:
    exp = dill.load(f)

In [22]:
methods = ["TOPSIS", "COPRAS", "PROMETHEE_II", "ARAS", "COCOSO", "CODAS", "EDAS", "MABAC"]
for i, method in enumerate(methods):
    print(f"{method}:\n")
    display(exp.results[i])
    wca = count_worst_case_avoidances(exp.results[i], [False, True, True], 1)
    print(f"Worst case avoidances:\n\t- for all metrics: {wca[0]}\n\t- for 2/3 metrics: {wca[1]}")
    print("AVG:")
    display(get_expconfig_mean_results(exp, i))
    print("\n")
    print("Avg rank:")
    display(get_average_metric_rank(exp.results[i], [False, True, True]))

TOPSIS:



[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             50.855054              0.789589           0.177056
 ShapTabularTreeWrapper  48.435592              0.959971           0.112136
 AnchorWrapper           39.174701              0.624701           0.204853
 AggregateExplainer      49.157089              0.905828           0.024381,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             50.662770              0.825587           0.081309
 ShapTabularTreeWrapper  52.936662              0.985788           0.590812
 AnchorWrapper           46.217694              0.503333           0.488336
 AggregateExplainer      59.522726              0.896811           0.207964,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             51.422699              0.816826           0.459738
 ShapTabularTreeWrapper  46.595295              0.978223           0.405122
 AnchorWra

Worst case avoidances:
	- for all metrics: 3
	- for 2/3 metrics: 5
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,50.939370,0.900119,0.209891
AnchorWrapper,42.336055,0.501847,0.375673
LimeWrapper,52.284901,0.804971,0.210756
ShapTabularTreeWrapper,52.550035,0.969277,0.310578




Avg rank:


...

[                              nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             50.910796              0.846004           0.007493
 ShapTabularTreeWrapper  48.435592              0.975285           0.006081
 AnchorWrapper           45.971281              0.516788           0.011108
 AggregateExplainer      48.147170              0.351833           0.082479,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             52.261707              0.833248           0.739161
 ShapTabularTreeWrapper  52.936662              0.988505           0.293668
 AnchorWrapper           45.971281              0.411390           0.128295
 AggregateExplainer      53.871222              0.367595           0.452462,
                               nrc  sensitivity_spearman  faithfulness_corr
 LimeWrapper             49.250940              0.837207           0.537476
 ShapTabularTreeWrapper  46.595295              0.974734           0.830150
 AnchorWra

Worst case avoidances:
	- for all metrics: 0
	- for 2/3 metrics: 3
AVG:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,50.826674,0.340002,0.204152
AnchorWrapper,44.517702,0.429388,0.255028
LimeWrapper,50.161282,0.842705,0.327493
ShapTabularTreeWrapper,52.550035,0.974389,0.365272




Avg rank:


,nrc,sensitivity_spearman,faithfulness_corr
AggregateExplainer,3.0,4.0,2.6
AnchorWrapper,1.0,3.0,2.2
LimeWrapper,2.8,2.0,2.6
ShapTabularTreeWrapper,3.2,1.0,2.6


### Discussion

TOPSIS had the second best worst case avoidance, avoiding the worst case for all metrics in 3/5 of the instances.
EDAS has the best worst case avoidance, avoiding the worst case for all metrics in 4/5 of the instances. It even had a better average faithfulness than TOPSIS.
All the other methods had poor performance.

# RAE-T vs. RAE-E | 10 samples
### Execution

In [6]:
results, metadata = evaluate_aggregate_explainer(
    clf, X_train, X_test, categorical_features,
    metrics_sets=[['nrc', 'sensitivity_spearman', 'rb_faithfulness_corr']],
    mcdm_algs=[pymcdm.methods.TOPSIS(), pymcdm.methods.EDAS()],
    n_instances=10,
    mp_jobs=5
)

metadata["description"] = "RAE-T vs RAE-S, 10 samples"

with open('pickles/pakdd2010/RAE-T_vs_RAE-S_10-allrank.pkl', 'wb') as f:
    dill.dump(ExperimentRun(metadata, results), f)

Selected indexes: [46170 11153 45986 29782 32648  4383 29772 27406 15446 37806]
Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2734 - val_loss: 1.1088
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - loss: 1.2625 - val_loss: 1.0645
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 580us/step - loss: 1.1651 - val_loss: 1.0158
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step - loss: 1.1171 - val_loss: 0.9676
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - loss: 1.1605 - val_loss: 0.9265
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 685us/step - loss: 0.9405 - val_loss: 0.8938
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 601us/step - loss: 1.0169 - val_loss: 0.8686
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - loss: 1.0262 - val_loss: 0.8491
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - loss: 0.9061 - val_loss: 0.8329
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step - loss: 0.8990 - val_loss: 0.8200
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 609us/st

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

	 Running instance 11153
	 Running instance 45986
	 Running instance 29782
	 Running instance 32648
	 Running instance 4383
	 Running instance 29772
	 Running instance 27406
	 Running instance 15446
	 Running instance 37806
Running evaluation for settings 2/2
Explainer components: [<class 'xai_agg.explainers.LimeWrapper'>, <class 'xai_agg.explainers.ShapTabularTreeWrapper'>, <class 'xai_agg.explainers.AnchorWrapper'>], Metrics: ['nrc', 'sensitivity_spearman', 'rb_faithfulness_corr'], MCDM algorithm: <pymcdm.methods.edas.EDAS object at 0x7394a278faf0>, Aggregation algorithm: wsum
	 Running instance 46170


Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

	 Running instance 11153
	 Running instance 45986
	 Running instance 29782
	 Running instance 32648
	 Running instance 4383
	 Running instance 29772
	 Running instance 27406
	 Running instance 15446
	 Running instance 37806


### Analysis

In [6]:
with open('pickles/pakdd2010/RAE-T_vs_RAE-S_10-allrank.pkl', 'rb') as f:
    exp = dill.load(f)

In [7]:
methods = ["RAE-T", "RAE-E"]
present_experiment_run(exp, labels=methods)

RAE-T:



[                              nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             51.630644              0.803886              0.419869
 ShapTabularTreeWrapper  52.493850              0.985127              0.212916
 AnchorWrapper           48.536659              0.663117              0.299670
 AggregateExplainer      40.574801              0.914040              0.386173,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             47.898890              0.821994              0.634467
 ShapTabularTreeWrapper  53.928645              0.973926              0.949856
 AnchorWrapper           47.930255              0.724737              0.782836
 AggregateExplainer      42.837433              0.920198              0.264767,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             54.227807              0.832478              0.529310
 ShapTabularTreeWrapper  46.543418              0.

Worst case avoidances:
	- for all metrics: 6
	- for 2/3 metrics: 10
AVG:


,nrc,sensitivity_spearman,rb_faithfulness_corr
AggregateExplainer,41.925472,0.901234,0.311135
AnchorWrapper,44.430640,0.559334,0.360829
LimeWrapper,52.019369,0.831510,0.474263
ShapTabularTreeWrapper,51.697464,0.975171,0.506293




Avg rank:


...

[                              nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             58.629704              0.817339              0.263984
 ShapTabularTreeWrapper  52.493850              0.989093              0.254924
 AnchorWrapper           47.052929              0.589602              0.193248
 AggregateExplainer      40.428748              0.888820              0.422227,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             52.658848              0.783468              0.573709
 ShapTabularTreeWrapper  53.928645              0.972604              0.633346
 AnchorWrapper           66.634771              0.694826              0.672515
 AggregateExplainer      40.819863              0.902163              0.740118,
                               nrc  sensitivity_spearman  rb_faithfulness_corr
 LimeWrapper             50.145239              0.859897              0.566481
 ShapTabularTreeWrapper  46.543418              0.

Worst case avoidances:
	- for all metrics: 5
	- for 2/3 metrics: 10
AVG:


,nrc,sensitivity_spearman,rb_faithfulness_corr
AggregateExplainer,41.146163,0.888688,0.292339
AnchorWrapper,46.921853,0.559957,0.417433
LimeWrapper,50.854801,0.828021,0.423305
ShapTabularTreeWrapper,51.697464,0.975633,0.456175




Avg rank:


,nrc,sensitivity_spearman,rb_faithfulness_corr
AggregateExplainer,1.4,2.2,3.1
AnchorWrapper,2.0,4.0,2.2
LimeWrapper,3.2,2.8,2.5
ShapTabularTreeWrapper,3.4,1.0,2.2
